In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [ ]:
#GRADER: CHANGE THESE PARAMETERS
ZIPFILENAME= "flowers_fixed.zip"
EXTRACTDIRNAME= "extracteddata"
FOLDERNAME= 'flowers_fixed'

In [ ]:
#mounting from drive
from zipfile import ZipFile

with ZipFile("/content/drive/MyDrive/flowers_fixed.zip", 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall(EXTRACTDIRNAME)

In [ ]:
PATH = os.path.join(EXTRACTDIRNAME, FOLDERNAME)
print(PATH)

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (256, 256)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

extracteddata/flowers_fixed
Found 8762 files belonging to 13 classes.


In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Found 4085 files belonging to 13 classes.


In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 103
Number of test batches: 25


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal')
])

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
#rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

print(type(preprocess_input))

<class 'function'>


In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
print(IMG_SHAPE)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

(256, 256, 3)
9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)
print(label_batch.shape)

(32, 8, 8, 1280)
(32,)


In [ ]:
base_model.trainable = False


In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [ ]:
#prediction_layer = tf.keras.layers.Dense(1)
prediction_layer = tf.keras.layers.Dense(13,activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 13)


In [ ]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(loss = 'sparse_categorical_crossentropy', optimizer= 'adam', metrics=['sparse_categorical_accuracy'])


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 256, 256, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 256, 256, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                             

In [ ]:
len(model.trainable_variables)

2

In [ ]:
#initial accuracy
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_dataset)

103/103 [==============================] - 17s 120ms/step - loss: 2.7110 - sparse_categorical_accuracy: 0.1148


In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 2.71
initial accuracy: 0.11


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping =  EarlyStopping(
    monitor='val_sparse_categorical_accuracy', 
    patience=1, 
)

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset,
                    callbacks=[early_stopping])

Epoch 1/10
274/274 [==============================] - 48s 159ms/step - loss: 0.7868 - sparse_categorical_accuracy: 0.7715 - val_loss: 0.5197 - val_sparse_categorical_accuracy: 0.8478
Epoch 2/10
274/274 [==============================] - 43s 158ms/step - loss: 0.3494 - sparse_categorical_accuracy: 0.8971 - val_loss: 0.4351 - val_sparse_categorical_accuracy: 0.8627
Epoch 3/10
274/274 [==============================] - 43s 157ms/step - loss: 0.2726 - sparse_categorical_accuracy: 0.9190 - val_loss: 0.3997 - val_sparse_categorical_accuracy: 0.8715
Epoch 4/10
274/274 [==============================] - 43s 157ms/step - loss: 0.2375 - sparse_categorical_accuracy: 0.9289 - val_loss: 0.3816 - val_sparse_categorical_accuracy: 0.8782
Epoch 5/10
274/274 [==============================] - 43s 157ms/step - loss: 0.2112 - sparse_categorical_accuracy: 0.9354 - val_loss: 0.3824 - val_sparse_categorical_accuracy: 0.8764


In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  154


In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['sparse_categorical_accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 256, 256, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 256, 256, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                             

In [ ]:
len(model.trainable_variables)

56

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset,
                         callbacks=[early_stopping])

Epoch 5/20
274/274 [==============================] - 67s 218ms/step - loss: 0.1639 - sparse_categorical_accuracy: 0.9434 - val_loss: 0.3446 - val_sparse_categorical_accuracy: 0.8932
Epoch 6/20
274/274 [==============================] - 59s 214ms/step - loss: 0.1281 - sparse_categorical_accuracy: 0.9573 - val_loss: 0.3290 - val_sparse_categorical_accuracy: 0.8959
Epoch 7/20
274/274 [==============================] - 59s 213ms/step - loss: 0.1011 - sparse_categorical_accuracy: 0.9699 - val_loss: 0.3322 - val_sparse_categorical_accuracy: 0.9023
Epoch 8/20
274/274 [==============================] - 58s 212ms/step - loss: 0.0850 - sparse_categorical_accuracy: 0.9731 - val_loss: 0.3447 - val_sparse_categorical_accuracy: 0.8974


In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

25/25 [==============================] - 3s 112ms/step - loss: 0.2949 - sparse_categorical_accuracy: 0.9062
Test accuracy : 0.90625


In [ ]:
!mkdir saved_models

mkdir: cannot create directory ‘saved_models’: File exists


In [ ]:
#fine tuned model is finished training, save to file
model.save("saved_models/mymodel.h5")

In [ ]:
 !zip -r final.zip saved_models

updating: saved_models/ (stored 0%)
updating: saved_models/mymodel.h5 (deflated 10%)


In [ ]:
print(test_dataset)